In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageSequence
import time
import pickle
import imageio
from utils.import_data import *
from utils.functions_calcium_analysis import *

In [2]:
fishlabel = '190104_F2'
trial = '8'
output_path = '/network/lustre/iss01/wyart/analyses/2pehaviour/ML_pipeline_output/'

In [3]:
bouts = load_bout_object(output_path, fishlabel, trial)
exp = load_experiment(output_path, fishlabel)[0]
struct= load_output_struct(fishlabel, trial, output_path+'dataset/', '/network/lustre/iss01/wyart/analyses/2pehaviour/suite2p_output/')
TA = struct['TA_all']

In [4]:
tif_path = '/network/lustre/iss01/wyart/analyses/2pehaviour/suite2p_output/190104_F2/8/suite2p/plane0/reg_tif/'

# loading the tif image
im1 = Image.open(tif_path + 'file000_chan0.tif')
im2 = Image.open(tif_path + 'file001_chan0.tif')

# converting the tiff image in array
imarray1 = np.array(im1)
imarray2 = np.array(im2)

nFrames = 846

# the tiff array will be in 3D: 2D of number of pixels, and the third D is the time.
tiff_array = np.zeros((imarray1.shape[0], imarray1.shape[1], nFrames))

In [5]:
# get each frame of the tif file to be concatenated to the array
for i, frame in enumerate(ImageSequence.Iterator(im1)):
    tiff_array[:,:,i] = frame
    
for j, frame in enumerate(ImageSequence.Iterator(im2)):
    tiff_array[:,:,i+j] = frame

In [6]:
# get baseline
baseline_array = tiff_array[:,:,1:20]

baseline_value = np.zeros(tiff_array.shape)

for i in range(tiff_array.shape[0]):
    for j in range(tiff_array.shape[1]):
        baseline_value[i,j,:] = np.median(baseline_array[i,j,:])


In [7]:
dff_array = (tiff_array/baseline_value)*1000

/home/mathilde.lapoix/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning:

divide by zero encountered in true_divide

/home/mathilde.lapoix/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning:

invalid value encountered in true_divide



In [8]:
# save dff array as an image
dff_video = []
for array in range(846):
    frame = Image.fromarray(dff_array[:,:,array])
    dff_video.append(frame)


In [9]:
imageio.mimsave('test_dff_190104_F2_8.avi', dff_video, format='avi')

ImportError: To use the imageio ffmpeg plugin you need to 'pip install imageio-ffmpeg'

In [ ]:
def cut_calcium_video(bout, bouts, dff_array, exp, output_path):
    fps_2p = exp.fps_2p
    fps_beh = exp.fps_beh
    start = math.floor((bouts[bout].start/fps_beh)*fps_2p)
    end = start + 2*fps_2p
    frames = np.arange(start, end+1)
    output_video = []
    for i in frames:
        output_frame = Image.fromarray(dff_array[:,:,i])
        output_video.append(output_frame)
    imageio.mimsave('calcium_imaging.avi', o, format='avi')